# Face Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect face landmarks from images.

# Preparation

In [36]:
# !wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

## Visualization utilities

In [37]:
#@markdown We implemented some functions to visualize the face landmark detection results. <br/> Run the following cell to activate the functions.
import mediapipe as mp # type: ignore
from mediapipe import solutions # type: ignore
from mediapipe.framework.formats import landmark_pb2 # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import time # type: ignore

def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()


last_is_blink = False
last_doze_time = 0
last_wake_time = 0
doze_counter = 0
doze_timer = 0
doze_start = 0
is_doze = False
def print_eyes_status(face_blendshapes):
  global last_is_blink
  global last_doze_time
  global last_wake_time
  global doze_counter
  global doze_timer
  global doze_start
  global is_doze
  face_blendshapes_scores = np.array([face_blendshapes_category.score for face_blendshapes_category in face_blendshapes])
  upward_score = face_blendshapes_scores[17] + face_blendshapes_scores[18]
  downward_score = face_blendshapes_scores[11] + face_blendshapes_scores[12]
  leftward_score = face_blendshapes_scores[14] + face_blendshapes_scores[15]
  rightward_score = face_blendshapes_scores[13] + face_blendshapes_scores[16]
  dir_score = np.array([["upward",upward_score], ["downward",downward_score], ["leftward",leftward_score], ["rightward",rightward_score]])
  dir = dir_score[np.argmax(dir_score[:,1])][0]
  #print(dir)
  is_blink = face_blendshapes_scores[9]>0.6 and face_blendshapes_scores[10]>0.6
  if is_blink:
    if not last_is_blink:
      last_doze_time = time.time()
    if last_doze_time != 0 and time.time() > last_doze_time:
      doze_counter += 1
      time.sleep(0.01)
    if doze_counter > 70:
      if not is_doze:
        doze_start = time.time()
      is_doze = True
      last_wake_time = 0

  else:
    if last_is_blink:
      last_wake_time = time.time()
    if last_wake_time != 0 and time.time() > last_wake_time:
      doze_counter -= 1
      time.sleep(0.01)
    if doze_counter < 20:
      doze_counter = 0
      is_doze = False
  doze_timer = time.time() - doze_start    
  last_is_blink = is_blink
  #print(is_blink)
  if is_doze:
    print(f"doze counter {doze_counter}",f"dozing for {doze_timer:.2f}s")
  else:
    print(f"{'blinking' if is_blink else f'looking {dir}'}")
  
    

## Running inference and visualizing the results

Here are the steps to run face landmark detection using MediaPipe.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python) to learn more about configuration options that this task supports.


In [38]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

In [39]:
import cv2 # type: ignore
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# print(ret_cam)
while True:
    _, frame = cap.read()
    if frame is None:
        continue
    cam_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)   
    detection_result_cam = detector.detect(cam_image)
    if len(detection_result_cam.face_blendshapes)==0 or len(detection_result_cam.face_landmarks)==0:
        continue
    annotated_image = draw_landmarks_on_image(cam_image.numpy_view(), detection_result_cam)
    cv2.namedWindow('cam', cv2.WINDOW_AUTOSIZE)
    cv2.imshow("cam", annotated_image)
    print_eyes_status(detection_result_cam.face_blendshapes[0])
    if cv2.waitKey(5) & 0xFF == ord('q'): 
        print("Exiting...")          
        cap.release()
        cv2.destroyAllWindows()
        break

looking leftward
looking leftward
looking leftward
looking leftward
looking leftward
looking leftward
looking leftward
looking leftward
looking leftward
looking leftward
looking leftward
looking leftward
looking downward
looking rightward
looking rightward
looking rightward
looking rightward
looking upward
looking upward
looking upward
looking upward
looking downward
looking downward
looking downward
looking downward
looking leftward
looking leftward
looking upward
looking upward
looking upward
looking upward
looking upward
looking downward
looking downward
blinking
blinking
blinking
blinking
blinking
blinking
blinking
looking downward
looking downward
blinking
blinking
blinking
blinking
looking downward
looking upward
looking upward
Exiting...


We will also visualize the face blendshapes categories using a bar graph.

And print the transformation matrix.

In [40]:
#(detection_result.facial_transformation_matrixes)